In [1]:
# Librerías externas de Python
import time
import numpy as np
import pandas as pd
from datetime import datetime

In [52]:
def crea_BD(filename):
    # Crea una base de datos vacía con la estructura correcta
    
    hdf_db = pd.HDFStore(filename,'a',complib="zlib",complevel=4)
    hdf_db.put('data/tabla',pd.DataFrame(columns=['Pollo', 'Medida',
                                                       'Freq',  'Z_mod', 'Z_Fase','Err', 'Eri' ,'E_mod',
                                                       'E_fase','R'    ,'X']), format='table')
    hdf_db.put('data/pollos_estado',pd.DataFrame(columns=['Pollo','Medida',
                                                          'Fecha','Estado','Primero','Ultimo']),format='table')
    hdf_db.put('data/fit',pd.DataFrame( columns=['Pollo','Medida','A', 'B1', 'C1', 'D1',
                                                                       'B2', 'C2', 'D2',
                                                                       'B3', 'C3', 'D3',
                                                                  'Ae','B1e','C1e','D1e',
                                                                       'B2e','C2e','D2e',
                                                                       'B3e','C3e','D3e',
                                                 'EPS_INF','EPS_ALFA1','EPS_ALFA2','EPS_ALFA3',
                                                 'F_ALFA1','F_ALFA2','F_ALFA3',
                                                 'EPS_INFe','EPS_ALFA1e','EPS_ALFA2e','EPS_ALFA3e',
                                                 'F_ALFA1e','F_ALFA2e','F_ALFA3e','R2']), format='table')
    # A partir de los duplicados en 'Pollo' y 'Medida' de pollos_estado podemos reconstruir la BD si hay un
    # error de numeración, simplemente borrando la entrada o renombrando 'Pollo'o 'Medida'
    hdf_db.close()
 

def llena_BD(n_pollos,bueno_malo,excel,filename,pollo_inicial):
    # Llena la base de datos con datos de una excel primigenia
    tabla = pd.DataFrame(columns=['Pollo','Medida','Freq','Z_mod','Z_Fase','Err','Eri','E_mod','E_fase','R','X'])
    pollos_estado = pd.DataFrame(columns=['Pollo','Medida','Fecha','Estado','Primero','Ultimo'])
    
    hdf_db = pd.HDFStore(filename,'a',complib="zlib",complevel=4)
    
    for pollo in range(0,n_pollos):
        for medida in range(0,n_medidas):
            indice_maestro = pollo*n_medidas+medida
            # Creacion del indice para recorrer la Excel
            if (pollo == 0) & (medida == 0):
                indice = ''
            else:
                indice = '.'+str(indice_maestro)

            data_aux = np.concatenate([
                    np.ones((n_freq,1),dtype=int)*(pollo + pollo_inicial),np.ones((n_freq,1),dtype=int)*(medida),
                    np.reshape(np.array(excel['Freq'][0:n_freq],dtype=float),(-1,1)),
                    np.reshape(np.array(excel['Z_mod' + indice][0:n_freq],dtype=float),(-1,1)),
                    np.reshape(np.array(excel['Z_Fase' + indice][0:n_freq],dtype=float),(-1,1)),
                    np.reshape(np.array(excel['Err' + indice][0:n_freq],dtype=float),(-1,1)), 
                    np.reshape(np.array(excel['Eri' + indice][0:n_freq],dtype=float),(-1,1)),
                    np.reshape(np.array(excel['E_mod' + indice][0:n_freq],dtype=float),(-1,1)),
                    np.reshape(np.array(excel['E_f' + indice][0:n_freq],dtype=float),(-1,1)),
                    np.reshape(np.array(excel['R' + indice][0:n_freq],dtype=float),(-1,1)),
                    np.reshape(np.array(excel['             X' + indice][0:401],dtype=float),(-1,1))
                    ], axis=1)
            data_aux_df = pd.DataFrame(data_aux,columns=['Pollo','Medida','Freq','Z_mod',
                                                'Z_Fase','Err','Eri','E_mod','E_fase','R','X'])
            #tabla = tabla.append(data_aux_df, ignore_index=True)

            Primero = (indice_maestro + pollo_inicial * n_medidas) * n_freq
            Ultimo  = (indice_maestro + pollo_inicial * n_medidas) * n_freq + n_freq - 1
            pollo_aux = np.array([pollo + pollo_inicial,medida,fecha_hora,bueno_malo,Primero,Ultimo]).reshape(1,-1)
            pollo_aux_df = pd.DataFrame(pollo_aux, columns=['Pollo','Medida','Fecha','Estado','Primero','Ultimo'])
            #pollos_estado = pollos_estado.append(pollo_aux_df,ignore_index=True)         

            hdf_db.append('data/tabla', data_aux_df)
            hdf_db.append('data/pollos_estado', pollo_aux_df)
    
    hdf_db.close()

    
    
def lee_medida_BD(filename, pollo, medida):
    # Devuelve los datos de la tabla correspondientes con el pollo y la medida en formato DataFrame
    # extracto muestra las medidas "erroneas" con identificación de pollo y medida repetidos
    hdf_db = pd.HDFStore(filename,complib="zlib",complevel=4)
    p_e  = hdf_db.get('data/pollos_estado')
    t    = hdf_db.get('data/tabla')
    extracto = p_e[(p_e['Pollo']==str(pollo))&(p_e['Medida']==str(medida))]
    Primero = extracto['Primero'].to_numpy(dtype='int')[0]
    Ultimo  = extracto['Ultimo'].to_numpy(dtype='int')[0] 
    hdf_db.close()
    
    return t.iloc[Primero:Ultimo+1],extracto


def lee_fit_BD(filename, pollo, medida, repeticion=0):
    # Devuelve los datos del fit correspondientes con el pollo y la medida en formato DataFrame
    # Si hay más de un fit repetición selecciona el fit
    
    hdf_db = pd.HDFStore(filename,complib="zlib",complevel=4)
    f    = hdf_db.get('data/fit')
    extracto = f[(f['Pollo']==pollo)&(f['Medida']==medida)]
    hdf_db.close()
    return extracto.iloc[repeticion],extracto


def escribe_medida_BD(filename, pollo, medida, datos, estado):
    # Introduce una medida en la base de datos. Los datos se añaden a la tabla sin borrar nada 
    # (la tabla crece indefinidamente) y se crea una nueva entrada en pollos_estado sin comprobar la
    # existencia de pollos o medidas coincidentes. 
    # Si en caso de que la BD se corrompa por introducir entradas coincidentes en pollo y/o medida
    # se puede recuperar eliminando manualmente una de las entradas o reasignando la numeración del pollo
    # El número de medidas por pollo o pollos es totalmente arbitrario
    
    fecha_hora = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    hdf_db = pd.HDFStore(filename,'a',complib="zlib",complevel=4)
    n_freq = np.shape(datos)[0]
    
    datos_aux = np.concatenate([np.ones((n_freq,1),dtype=int)*(pollo),
                                np.ones((n_freq,1),dtype=int)*(medida),
                               datos],axis=1)
                    
    data_aux_df = pd.DataFrame(datos_aux,columns=['Pollo','Medida','Freq','Z_mod',
                                                'Z_Fase','Err','Eri','E_mod','E_fase','R','X'])
    t    = hdf_db.get('data/tabla')
    Primero = len(t)
    Ultimo  = Primero + n_freq - 1
    pollo_aux = np.array([pollo,medida,fecha_hora,estado,Primero,Ultimo]).reshape(1,-1)
    pollo_aux_df = pd.DataFrame(pollo_aux, columns=['Pollo','Medida','Fecha','Estado','Primero','Ultimo'])
    
    hdf_db.append('data/tabla', data_aux_df)
    hdf_db.append('data/pollos_estado', pollo_aux_df)

In [53]:
fit_n,extracto=lee_fit_BD("test_table.hdf",20,1,1)

In [54]:
extracto

,Pollo,Medida,A,B1,C1,D1,B2,C2,D2,B3,...,F_ALFA2,F_ALFA3,EPS_INFe,EPS_ALFA1e,EPS_ALFA2e,EPS_ALFA3e,F_ALFA1e,F_ALFA2e,F_ALFA3e,R2
0,20.0,1.0,0.741482,0.254251,1.0,0.170946,0.061586,5.922005,0.676183,0.0,...,835611.928511,0.0,1.601337,6.702495,19.405395,0.0,82.450049,9.523954e+05,0.0,0.856422
0,20.0,1.0,0.741474,0.279162,0.0,0.162095,0.060671,5.925843,0.696244,0.0,...,843029.306226,0.0,inf,NaN,NaN,0.0,inf,inf,0.0,0.855952


In [58]:
medida,extracto=lee_medida_BD("test_table.hdf",20,0)

In [59]:
medida

,Pollo,Medida,Freq,Z_mod,Z_Fase,Err,Eri,E_mod,E_fase,R,X
0,20.0,0.0,1.000000e+01,118.187351,2.215093,-1.076663e+08,-8.088362e+07,1.346633e+08,2.497296,-70.987590,94.493450
1,20.0,0.0,1.778279e+01,56.959287,-1.019773,1.338720e+08,8.226625e+07,1.571287e+08,-0.551024,29.821580,-48.528690
2,20.0,0.0,3.162278e+01,86.126670,-2.148633,4.894892e+07,-3.191866e+07,5.843627e+07,0.577837,-47.043520,-72.143680
3,20.0,0.0,5.623413e+01,51.941693,0.728248,-3.626547e+07,4.066695e+07,5.448840e+07,-2.299045,38.766240,34.570480
4,20.0,0.0,1.000000e+02,18.544819,0.974780,-7.102424e+07,4.817608e+07,8.582178e+07,-2.545576,10.410140,15.347290
5,20.0,0.0,1.778279e+02,26.153003,1.709843,-3.389118e+07,-4.743059e+06,3.422146e+07,3.002546,-3.624779,25.900590
6,20.0,0.0,3.162278e+02,4.990186,-3.106801,3.508210e+06,-1.007954e+08,1.008564e+08,1.536005,-4.987166,-0.173580
7,20.0,0.0,5.623413e+02,65.544656,2.921972,-9.407187e+05,-4.214284e+06,4.318002e+06,1.790417,-63.970280,14.279540
8,20.0,0.0,1.000000e+03,10.320572,1.249446,-1.463172e+07,4.870738e+06,1.542114e+07,-2.820242,3.259734,9.792259
9,20.0,0.0,1.778279e+03,8.714324,-2.250392,7.988578e+06,-6.454710e+06,1.027038e+07,0.679595,-5.476764,-6.778237


In [60]:
extracto

,Pollo,Medida,Fecha,Estado,Primero,Ultimo
0,20,0,19/02/2021 17:08:15,0,24060,24084
